In [1]:
import pandas as pd
import nltk
import gzip
import re
import os
from nltk.tokenize import word_tokenize

In [2]:
os.getcwd()
pd.set_option('display.max_colwidth', None)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rafael.DePabloVarona\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
df =pd.read_csv('./stackexchange_812k.csv.gz',compression="gzip")
df.columns

Index(['post_id', 'parent_id', 'comment_id', 'text', 'category'], dtype='object')

In [4]:
matches = dict()
matches["html_tags"] ="<[^>]*>"
matches["latex_expressions"] ="\$[^\$]*\$"
matches["digits"] = "[+-]?([0-9]*[.])?[0-9]+"
matches["return"] ="\\n"
matches["mentions"] ="@[\w]*"
matches["urls"] = "https?://[^(\s)]*\s"
matches["double quotes"] = '"'

In [22]:
def clean_text(row):
  text = row['text']
  patterns ="|".join(list(matches.values()))
  return re.sub(patterns,"", str(text))

def tokenizer_row(row):
  return word_tokenize(row['clean_text'])

def tokenizer_text(clean_text):
  return word_tokenize(clean_text)

def truncate_tokens(row,min_num_tokens=6,max_num_tokens=50):
    num_words = len(row['tokenize_clean_text'])
    if num_words < min_num_tokens:
        return ''
    if num_words > max_num_tokens:
        return row['tokenize_clean_text'][0:max_num_tokens]
    return row['tokenize_clean_text']

In [6]:
df['clean_text'] = df.apply(lambda row : clean_text(row), axis = 1)
df['tokenize_clean_text'] = df.apply(lambda row : tokenizer_text(row['clean_text']), axis = 1)
df['tokens'] = df.apply(lambda row : truncate_tokens(row), axis = 1)

In [9]:
path = "./clean_tokenized_stackexchange_812k.csv.zip"
df.to_csv(path,index=False,compression="zip")